submit:
1. Itamar Nakar
itamar.nakar@gmail.com
2. Nikita nedostoup
336495593
nikolos7771@gmail.com
3. Lirom Mishan
208199398
lyrwmmsn@gmail.com

In [33]:
# Hyper parameters
NEGATIVE_SAMPLES = 9  # Negative samples per pair
WINDOW_SIZE = 3  # Window size for context words

### Read dataset

import data by using pandas

In [34]:
import pandas as pd 

path_to_df = r"C:\Users\avim0\OneDrive - Bar-Ilan University - Students\Notability\שנה ג סמסטר ב\למידת מכונה\פרוייקט סופי\review_230k_.parquet"
df = pd.read_parquet(path_to_df)
df.head(10)

,title,text
0,"Truly is ""Jewel of the Upper Wets Side""",Stayed in a king suite for 11 nights and yes i...
1,My home away from home!,"On every visit to NYC, the Hotel Beacon is the..."
2,Great Stay,This is a great property in Midtown. We two di...
3,Modern Convenience,The Andaz is a nice hotel in a central locatio...
4,Its the best of the Andaz Brand in the US....,I have stayed at each of the US Andaz properti...
5,"One of NYC's Best Hotels, Hands Down",Excellent staff (they remembered our names fro...
6,Apartment living for less than most 5 stars,"I stayed at the Setai for 3 nights last week, ..."
7,Lovely stay at The Chatwal,My husband and I stayed at The Chatwal for 9 n...
8,Exclusive boutique hôtel,Wonderful boutique hotel located next to Times...
9,A Nice Stay for NYC!,This hotel is a nice stay for NYC because the ...


This part combine the title and the text together so the text will followed by the title in every row

In [35]:

def create_dataset(data: pd.DataFrame):
    """create our dataset"""

    # Combine 'title' and 'text' columns into a single 'combined' column
    data['combined'] = data['title'] + '. ' + data['text']

    processed_data = data[['combined']]

    processed_data.reset_index(drop=True, inplace=True)

    processed_data['combined'] = processed_data['combined'].str.split('.')

    # Use explode to create a new row for each sentence
    processed_data = processed_data.explode('combined')

    # Remove spaces from the sentences
    processed_data['combined'] = processed_data['combined'].str.strip()

    # Drop empty rows 
    processed_data = processed_data[processed_data['combined'] != '']
    # Reset index
    processed_data.reset_index(drop=True, inplace=True)

    return processed_data

dataset = create_dataset(df)
dataset.head(10)


C:\Users\avim0\AppData\Local\Temp\ipykernel_14096\3624023226.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  processed_data['combined'] = processed_data['combined'].str.split('.')


,combined
0,"Truly is ""Jewel of the Upper Wets Side"""
1,Stayed in a king suite for 11 nights and yes i...
2,Our room was on the 20th floor overlooking Bro...
3,Room was quite with no noise evident from the ...
4,It was great to be able to open windows when w...
5,"The beds, including the fold out sofa bed, wer..."
6,Wi-fi access worked like a dream with only one...
7,The location close to the 72nd Street subway s...
8,It is fabulous to have the kitchen with cookin...
9,This is the second time that members of the pa...


### clean and standardize the data

Delete foreign language Words, and it's punctuation
we ignored languages which are not english becuase we can't set rules for a lot of languages that have different rules.

In [36]:
import re
import nltk
from nltk.corpus import words

# Download the list of English words from NLTK
nltk.download('words')
english_vocab = set(words.words())

def clean_text(text):
    """
    Converts text to lowercase, removes punctuation (except $), and splits it into words.
    """
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'[^\w\s\$]', '', text)  # Remove punctuation except $
    return text.split()  # Split text into words

def filter_foreign_words(dataset):
    """
    Removes non-English words from the 'combined' column and stores the result in a new column 'english_only'.
    """
    def filter_non_english_words(text):
        words = clean_text(text)
        # Keep only English words and the $ symbol
        return ' '.join([word for word in words if word in english_vocab or word == '$'])

    # Create a new column 'english_only' with only English words
    dataset['combined'] = dataset['combined'].apply(filter_non_english_words)
    return dataset

# Apply the function to the dataset to remove foreign words
dataset = filter_foreign_words(dataset)

# Display the result with the new 'english_only' column
print(dataset['combined'].head())


[nltk_data] Downloading package words to
[nltk_data]     C:\Users\avim0\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


0                     truly is jewel of the upper side
1    stayed in a king suite for nights and yes it u...
2    our room was on the floor broadway and the mad...
3    room was quite with no noise evident from the ...
4    it was great to be able to open when we fresh ...
Name: combined, dtype: object


Replace rare words with the token '$'

In [ ]:
import re
import nltk
from nltk.corpus import words
from collections import Counter

def clean_text(text):
    """
    Converts text to lowercase, removes punctuation (except $), and splits it into words.
    """
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'[^\w\s\$]', '', text)  # Remove punctuation except $
    return text.split()  # Split text into words

def replace_rare_words_with_dollar(dataset, min_frequency=3):
    """
    Replaces rare English words in the 'combined' column with the $ symbol.
    """
    # Collect all words from the 'combined' column
    all_words = []
    for text in dataset['combined']:
        words = clean_text(text)
        all_words.extend(words)

    # Count word frequencies
    word_freq = Counter(all_words)

    # Replace rare words with the $ symbol
    def replace_rare_words(text):
        words = clean_text(text)
        # Keep words with frequency >= min_frequency, otherwise replace with $
        return ' '.join([word if word_freq[word] >= min_frequency or word == '$' else '$' for word in words])

    # Create a new column 'replaced_rare' with rare words replaced by $
    dataset['combined'] = dataset['combined'].apply(replace_rare_words)
    return dataset

min_frequency = 3  # Minimum frequency to keep a word

# Apply the function to replace rare words with the $ symbol
dataset = replace_rare_words_with_dollar(dataset, min_frequency=3)

# Display the result with the 'replaced_rare' column
print(dataset['combined'].head())


replace symbols by the token '$'

In [ ]:
import re

# Define a function to replace noisy words with '$'
def replace_noisy_words_in_row(text):
    words = text.split()  # Split the text into individual words
    return ' '.join([word if re.match(r'^[a-zA-Z]+$', word) else '$' for word in words])

# Apply the function to the 'combined' column in the dataset
dataset['combined'] = dataset['combined'].apply(replace_noisy_words_in_row)

# Display the first few rows
print(dataset['combined'].head())

In this script we replace different types of stop words with the token '$'. We use nltk.courpus that collect many different stpowords

In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Download necessary resources
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')  # for extended WordNet data

# Initialize stopwords set and lemmatizer
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def clean_and_normalize_text(text):
    """
    Cleans and normalizes the text by:
    - Lowercasing
    - Removing punctuation
    - Removing numbers
    - Replacing stopwords with '$'
    - Applying lemmatization
    """
    # Convert to lowercase
    text = text.lower()

    # Remove numbers
    text = re.sub(r'\d+', '', text)

    # Remove stopwords and apply lemmatization
    words = text.split()
    lemmatized_words = [
        lemmatizer.lemmatize(word) if word not in stop_words else '$'
        for word in words
    ]

    return ' '.join(lemmatized_words)  # Join the lemmatized words back into a single string

def prepare_and_normalize_dataset(dataset):
    """
    Applies the cleaning and normalization function to the entire dataset.
    """
    # Apply the cleaning function to each sentence in the 'combined' column
    dataset['combined'] = dataset['combined'].apply(lambda x: clean_and_normalize_text(x))
    return dataset

dataset = prepare_and_normalize_dataset(dataset)

# Output the cleaned and normalized dataset
print(dataset.head())


Here we generate word pairs, because some of the the pairs are compound nouns. for example 'bus station'. It helps us reduce the size of the library and improve the running time.

In [ ]:
import re
from collections import defaultdict
import pandas as pd


def generate_word_pairs(text, window_size):
    """
    Generate word pairs from the text within a given window size, skipping pairs containing '$'.
    param text: Input text
    param window_size: Number of words to consider around the target word
    return: List of word pairs
    """


    # Split into words
    words = text.split()

    # List to store word pairs
    word_pairs = []

    # Iterate over each word and create pairs with it's neighbors
    for idx, word in enumerate(words):
        if '$' in word:
            continue  # Skip if the target word contains '$'

        start = max(0, idx - window_size)
        end = min(len(words), idx + window_size + 1)

        # Create pairs: current word + its neighbors, skip if neighbor contains '$'
        for neighbor in words[start:idx] + words[idx+1:end]:
            if '$' in neighbor:
                continue  # Skip if the neighbor word contains '$'
            word_pairs.append((word, neighbor))

    return word_pairs

def identify_phrases(word_pairs, threshold=25):
    """
    Identify frequent word pairs (phrases) and return the modified text.

    param word_pairs: List of word pairs from the text
    param threshold: Frequency threshold to consider a word pair as a phrase
    return: Dictionary of word pairs that should be treated as phrases
    """
    # Count occurrences of word pairs
    pair_count = defaultdict(int)
    for pair in word_pairs:
        pair_count[pair] += 1

    # Select pairs that occur more than the threshold
    phrases = {pair: '_'.join(pair) for pair, count in pair_count.items() if count >= threshold}

    return phrases

def replace_with_phrases(text, phrases):
    """
    Replace frequent word pairs with phrases in the text.

    param text: Input text
    param phrases: Dictionary of word pairs and corresponding phrases
    return: Modified text with phrases
    """
    # Clean text (remove punctuation and lowercase, but keep $)
    text = re.sub(r'[^\w\s\$]', '', text.lower())

    # Replace frequent word pairs with phrases
    for (word1, word2), phrase in phrases.items():
        text = text.replace(f"{word1} {word2}", phrase)

    return text

# Batch processing with the updated way of skip '$'
def process_dataset_in_batches(dataset, batch_size=1024, window_size=1, threshold=25):
    total_rows = len(dataset)
    processed_data = pd.DataFrame()

    # Process dataset in batches
    for batch_start in range(0, total_rows, batch_size):
        batch = dataset.iloc[batch_start:batch_start + batch_size].copy()  # Get a batch of rows
        all_word_pairs = []

        # Generate word pairs for each text in the 'combined' column
        for text in batch['combined']:
            word_pairs = generate_word_pairs(text, window_size)
            all_word_pairs.extend(word_pairs)

        # Identify frequent word pairs and create phrases
        phrases = identify_phrases(all_word_pairs, threshold)

        # Replace frequent word pairs with phrases in each text
        batch['combined'] = batch['combined'].apply(lambda x: replace_with_phrases(x, phrases))

        # Append processed batch to final dataset
        processed_data = pd.concat([processed_data, batch], ignore_index=True)

    return processed_data

# print dataset after reduced library size
dataset = process_dataset_in_batches(dataset, batch_size=1024, window_size=1, threshold=25)

print(dataset.head())


calculate words frequencies of apperance

In [ ]:
import numpy as np
from collections import defaultdict

def calculate_word_frequencies(dataset):
    """
    Calculate the frequency of each word in the dataset.

    param dataset: Input DataFrame containing a 'combined' column with text
    return: Dictionary with word frequencies
    """
    word_count = defaultdict(int)

    # Iterate over each text in the dataset
    for text in dataset['combined']:
        words = text.split()
        for word in words:
            word_count[word] += 1

    return word_count

def subsample_frequent_words(dataset, t):
    """
    Apply subsampling to frequent words in the dataset, keeping words with '$'.

    param dataset: Input DataFrame containing a 'combined' column with text
    param t: Threshold for subsampling frequent words
    return: Modified dataset with subsampled text
    """
    # Calculate word frequencies across the dataset
    word_frequencies = calculate_word_frequencies(dataset)

    # Total number of words in the dataset
    total_words = sum(word_frequencies.values())

    # Calculate the probability of keeping each word
    word_probabilities = {}
    for word, count in word_frequencies.items():
        frequency = count / total_words
        word_probabilities[word] = 1 - np.sqrt(t / frequency) if frequency > t else 1.0

    # Apply subsampling to the 'combined' column
    def subsample_text(text):
        words = text.split()
        # Keep words with '$' and apply subsampling to others
        sampled_words = [
            word if ('$' in word or np.random.rand() < word_probabilities.get(word, 1.0))
            else '$'
            for word in words
        ]
        return ' '.join(sampled_words)

    # Apply the subsampling to the entire dataset
    dataset['combined'] = dataset['combined'].apply(subsample_text)

    return dataset

# the dataset with the text in the 'combined' column
dataset = subsample_frequent_words(dataset, t=1e-2)

# Output subsampling
print(dataset.head())


This code cleans all words that have only one letter

In [ ]:
import re

# Function to replace one-letter words (excluding 'a' and 'I') with '$'
def replace_one_letter_words(text):
    # Replace one-letter words with a dollar symbol using regex
    replaced_text = re.sub(r'\b(?![aI]\b)[a-zA-Z]\b', '$', text)
    # Replace multiple spaces with a single space
    replaced_text = re.sub(r'\s+', ' ', replaced_text).strip()
    return replaced_text

# Apply the function to the 'combined' column in the dataset
dataset['combined'] = dataset['combined'].apply(replace_one_letter_words)

# Display a sample of the cleaned data
print(dataset['combined'].head())


### Create A vocabulary

create a vocabulary from the words in the data

In [ ]:
def create_vocabulary(dataset):
    """
    Creates a vocabulary from unique words in the 'combined_normalized' column of the dataset.
    """
    unique_words = set()

    # Iterate over each row in the DataFrame
    for text in dataset['combined']:
        words = text.split()  # The text is already normalized, so just split into words
        unique_words.update(words)

    # Create a vocabulary dictionary that maps words to unique indexes
    unique_words_dict = {word: idx for idx, word in enumerate(unique_words)}

    return unique_words_dict

# Create the vocabulary from the normalized text
vocabulary = create_vocabulary(dataset)

# Print the first few rows of the dataset and the vocabulary size
print(dataset[['combined']].head())
print(f"Vocabulary size: {len(vocabulary)}")
dataset.head(10)

### Word encoding

encoding the words in the dataset to vectors using the vocabulary

In [ ]:
def encode_text(text, vocabulary):
    """
    Encodes a text based on the provided vocabulary.
    Any word not in the vocabulary will be skipped.
    """
    words = text.lower().replace(",", "").replace("!", "").split()
    return [vocabulary.get(word,  ) for word in words if word in vocabulary]

def encode_dataset(dataset, vocabulary):
    """
    Encodes the entire dataset using the provided vocabulary.
    Applies the encode_text function to each sentence in the 'combined' column.
    """
    # Apply the encode_text function to all rows in the 'combined' column
    dataset['encoded'] = dataset['combined'].apply(lambda x: encode_text(x, vocabulary))
    return dataset

# create the vocabulary
vocabulary = create_vocabulary(dataset)

# encode the entire dataset
encoded_dataset = encode_dataset(dataset , vocabulary)

# Print the first few rows
print(encoded_dataset[['combined', 'encoded']].head())


cleaning empty rows

In [ ]:
# the index of the $ symbol is known
encoded_dollar_sign = vocabulary.get('$')

# Find rows with empty or invalid values in the 'encoded' column
empty_or_invalid_rows_encoded = encoded_dataset[
    encoded_dataset['encoded'].isna() |  # Empty values
    (encoded_dataset['encoded'].apply(len) == 0) |  # Empty encoded lists
    (encoded_dataset['encoded'].apply(lambda x: len(x) == 1 and x[0] == encoded_dollar_sign))  # Only $ symbol
]

# Print the number of rows with empty or invalid encoded values
print(f"Number of rows with empty or invalid encoded values: {len(empty_or_invalid_rows_encoded)}")

# Show the first few rows with empty or invalid encoded values
print(empty_or_invalid_rows_encoded.head())

# Remove rows with empty or invalid encoded values
encoded_dataset = encoded_dataset.drop(empty_or_invalid_rows_encoded.index).reset_index(drop=True)

# Print the updated dataset size after cleaning
print(f"Dataset size after cleaning: {len(encoded_dataset)}")


### Generate Positive and Negative Pairs

In [ ]:
import numpy as np
import tensorflow as tf
from collections import defaultdict
def generate_training_data_with_batching(encoded_dataset, number_of_neg_samples: int, batch_size: int, window_size: int):
    """
    Generate positive and negative pairs using batching and optimized negative sampling.
    """
    word_freq = defaultdict(int)

    # Count the frequency of words, ignoring the `$` symbol
    for sentence in encoded_dataset:
        for word in sentence:
            if word != encoded_dollar_sign:  # Ignore $ when counting frequencies
                word_freq[word] += 1

    total_words = sum(word_freq.values())
    word_prob = np.array([(freq / total_words) ** 0.75 for freq in word_freq.values()])
    words = np.array(list(word_freq.keys()))
    word_prob /= word_prob.sum()

    all_targets, all_contexts, all_labels = [], [], []

    for batch_start in range(0, len(encoded_dataset), batch_size):
        batch_sentences = encoded_dataset[batch_start:batch_start + batch_size]

        batch_targets, batch_contexts = [], []

        for sentence in batch_sentences:
            sentence_length = len(sentence)

            for i, target_word in enumerate(sentence):
                if target_word == encoded_dollar_sign:
                    continue  # Skip $ as the target word

                # Form the full window (including $)
                start = max(i - window_size, 0)
                end = min(i + window_size + 1, sentence_length)

                # The context includes all words except the current target word
                context = list(sentence[start:i]) + list(sentence[i + 1:end])

                # Remove the $ symbol from the context
                context = [w for w in context if w != encoded_dollar_sign]

                # organize the context to the correct size if it get out of the limits of the window size after removing $
                context = context[:window_size]  # Keep only the nearest words

                # Add target-context pairs
                batch_targets.extend([target_word] * len(context))
                batch_contexts.extend(context)

        batch_targets = np.array(batch_targets)
        batch_contexts = np.array(batch_contexts)

        # Generate negative samples
        num_positive = len(batch_targets)
        negative_samples = np.random.choice(words, size=num_positive * number_of_neg_samples, p=word_prob)
        negative_samples = negative_samples.reshape(num_positive, number_of_neg_samples)

        # Create negative pairs
        target_neg = np.repeat(batch_targets, number_of_neg_samples)
        negative_words = negative_samples.flatten()

        pairs_positive = np.stack([batch_targets, batch_contexts], axis=1)
        pairs_negative = np.stack([target_neg, negative_words], axis=1)

        pairs = np.concatenate([pairs_positive, pairs_negative], axis=0)
        labels = np.concatenate([np.ones(len(pairs_positive), dtype=np.int32),
                                 np.zeros(len(pairs_negative), dtype=np.int32)])

        # Add results to global lists
        all_targets.append(pairs[:, 0])
        all_contexts.append(pairs[:, 1])
        all_labels.append(labels)

    # Convert global lists to arrays
    all_targets = np.concatenate(all_targets)
    all_contexts = np.concatenate(all_contexts)
    all_labels = np.concatenate(all_labels)

    return all_targets, all_contexts, all_labels

# using the function
number_of_neg_samples = 4
window_size = 2
batch_size = 16382

# `encoded_dataset` 
targets, contexts, labels = generate_training_data_with_batching(
    encoded_dataset['encoded'], number_of_neg_samples, batch_size, window_size
)

print(f"Total pairs generated: {len(targets)}")
print(f"Example target-context pairs and labels: {targets[:5]}, {contexts[:5]}, {labels[:5]}")


### Define the model

1. we build here the skip gram model
2. we define the input layers as targets and context words.
3. convert input words to vectors.
4. by dot product we calculate the relation between the vectors. 
5. In order to compile the model we use binary cross entropy and adam optimizer

In the output:
1. target_input an context_input are the input layers that represent the target word and the the context word.
2. in the embedding we give every word a one hot vector.
3. dot is the scalar multiply of vectors target_input and context_input
4. dropout is cut off some of the neurons for regularization.
5. dense is the output.

In [ ]:
import tensorflow as tf

# Create a model with Dropout and L2 regularization
def create_skipgram_model_with_regularization(vocabulary_size, embedding_dim, dropout_rate, l2_lambda):
    """
    Creates a Skip-Gram model with Dropout and L2 regularization.
    """
    target_input = tf.keras.Input(shape=(1,), name='target_input')
    context_input = tf.keras.Input(shape=(1,), name='context_input')

    embedding = tf.keras.layers.Embedding(
        input_dim=vocabulary_size,
        output_dim=embedding_dim,
        input_length=1,
        name='embedding'
    )

    target_embedding = embedding(target_input)
    context_embedding = embedding(context_input)

    # Compute the dot product between the two embeddings
    dot_product = tf.keras.layers.Dot(axes=-1)([target_embedding, context_embedding])

    # Apply Dropout for regularization
    dropout_output = tf.keras.layers.Dropout(dropout_rate)(dot_product) # Apply dropout to the dot product


    # Output layer with sigmoid activation and regulizacion
    output = tf.keras.layers.Dense(
    1, activation='sigmoid',
    kernel_regularizer=tf.keras.regularizers.l2(l2_lambda)
    )(dropout_output) # Pass the output of dropout to Dense layer


    model = tf.keras.Model(inputs=[target_input, context_input], outputs=output)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model

# Define model parameters
VOCABULARY_SIZE = len(vocabulary)
EMBEDDING_DIM = 220

# Create the model
skipgram_model = create_skipgram_model_with_regularization(
    vocabulary_size=VOCABULARY_SIZE,
    embedding_dim=EMBEDDING_DIM,
    dropout_rate=0.2,
    l2_lambda=0.01
)

# Save the model for future use
skipgram_model.save("skipgram_model_initial.h5")
print("Model created and saved.")
skipgram_model.summary()

### Training
train the model by use skip gram model

Here we are training and testing the model. we add early_stoping for stop the runing when the model is not improving anymore

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.optimizers import Adam

# Load the model
skipgram_model = tf.keras.models.load_model("skipgram_model_initial.h5", compile=False)
optimizer = Adam(learning_rate=0.01) # decrethe lerning rate for avoinding to strong changing in weigths
# Recompile the model to avoid issues with the optimizer
skipgram_model.compile(
    optimizer=optimizer,
    loss='binary_crossentropy',
    metrics=['accuracy']
  )

# Ensure that the data has the correct shape
targets = np.reshape(targets, (-1, 1))
contexts = np.reshape(contexts, (-1, 1))
labels = np.reshape(labels, (-1, 1))

# Check the shape of the data
print(f"Targets shape: {targets.shape}")
print(f"Contexts shape: {contexts.shape}")
print(f"Labels shape: {labels.shape}")

# Set up early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(
   monitor='val_loss',
   patience=5,
   restore_best_weights=True
)

# Train the model
history = skipgram_model.fit(
   [targets, contexts],
   labels,
   epochs=17,
   batch_size=32768,
   validation_split=0.2,

   callbacks=[early_stopping]
)

# Save the trained model
skipgram_model.save("word2vec_model_with_dropout_trained.h5")
print("Training completed and model saved.")
skipgram_model.summary()

__________________________NEW

### Save embeddings

save the embeddings to external payrrow map

In [ ]:
import pyarrow as pa
import pyarrow.parquet as pq
import numpy as np

def save_embeddings(model, vocabulary: list, path: str):
    """
    Save the embeddings to a Parquet file using pyarrow.

    param model: The trained TensorFlow model
    param vocabulary: List of tokens (words)
    param path: Path to save the embeddings file (without extension)

    The function will extract the embedding weights and save them in a PyArrow Table format:
    <word> <embedding_vector>
    """
    # Get the embedding layer weights
    embedding_layer = model.get_layer('embedding')
    embeddings = embedding_layer.get_weights()[0]  # The weights are in the first index of get_weights()

    # Ensure the embeddings and vocabulary size match
    if len(vocabulary) != embeddings.shape[0]:
        print(len(vocabulary))
        print((embeddings.shape[0]))
        raise ValueError("Vocabulary size and embeddings size do not match!")
        
    # Convert the embeddings to a list of lists (for each word)
    embedding_list = embeddings.tolist()

    # Create a PyArrow table with two columns: "word" and "embedding"
    table = pa.Table.from_pydict({
        "word": vocabulary,
        "embedding": embedding_list
    })

    # Save the table as a Parquet file
    pq.write_table(table, f"{path}.parquet")

    print(f"Embeddings saved to {path}.parquet")

save_embeddings(skipgram_model, vocabulary, f"word2vec_embedding_w{WINDOW_SIZE}_ns{NEGATIVE_SAMPLES}")


### Find Most similar
Note that this must be efficient

return list of the k most similar vectors to a word with regards to cosine_similarity

In [ ]:
from typing import Dict, List, Tuple 
import numpy as np

def cosine_similarity(vec1: np.ndarray, vec2: np.ndarray) -> float:
    """
    Compute the cosine similarity between two vectors.

    param: vec1: First vector (numpy array)
    param: vec2: Second vector (numpy array)
    return: Cosine similarity as a float
    """
    # Compute the dot product of the two vectors
    dot_product = np.dot(vec1, vec2)

    # Compute the L2 norms (magnitudes) of the vectors
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)

    # Compute the cosine similarity
    if norm_vec1 == 0 or norm_vec2 == 0:
        # If either vector has zero magnitude, cosine similarity is undefined
        return 0.0

    return dot_product / (norm_vec1 * norm_vec2)

def find_most_similar(model, word: str, vocabulary: Dict[str, int], k: int = 10) -> List[Tuple[str, float]]:
    """
    Find most similar tokens to the input word.

    param: model: the trained Word2Vec model
    param: word: the word to find the most similar words to
    param: vocabulary: dictionary mapping words to indexes
    param: k: number of most similar words to return
    return: List of tuples containing similar words and their cosine similarity scores
    """
    
    # Retrieve the embeddings from the model
    embedding_layer = model.get_layer('embedding') 
    embeddings = embedding_layer.get_weights()[0]  # (vocab_size, embedding_dim)

    # Get the index of the input word in the vocabulary
    if word not in vocabulary:
        raise ValueError(f"Word '{word}' not found in the vocabulary")

    word_idx = vocabulary[word]  # Get the index of the input word

    # Get the embedding of the input word
    word_embedding = embeddings[word_idx]  # Embedding of the input word

    # Compute cosine similarity between the input word and all other words
    similarities = []

    for other_word, other_idx in vocabulary.items():
        if other_word == word:
            continue  # Skip the input word itself
        other_embedding = embeddings[other_idx]
        similarity = cosine_similarity(word_embedding, other_embedding)
        similarities.append((other_word, similarity))

    # Sort the words by similarity in descending order
    most_similar = sorted(similarities, key=lambda x: x[1], reverse=True)

    # Return the top k most similar words
    return most_similar[:k]
putinig_word = 'hotel'
print(f"My word is: {putinig_word} ")
print(find_most_similar(skipgram_model, putinig_word, vocabulary))

### Dimentionality Reduction
visualize some clusters (pick some subset of words to show the labels for)

we will decrease the number of dimensions of the vectors

In [ ]:
embedding_layer = skipgram_model.get_layer('embedding')
X = embeddings = embedding_layer.get_weights()[0]
words = vocabulary

from sklearn.preprocessing import StandardScaler
X_scaled = StandardScaler().fit_transform(X)

from sklearn.decomposition import PCA
n_components = 2
pca = PCA(n_components=n_components)
X_reduced = pca.fit_transform(X_scaled)


In [ ]:
import matplotlib.pyplot as plt

# Initialize a defaultdict to count word frequencies
word_count = defaultdict(int)

# Iterate over each row in the dataset
for text in encoded_dataset['combined']:
    # Convert text to lowercase and remove punctuation
    words_in_text = text.lower().replace(",", "").replace("!", "").split()

    # Count occurrences of words that are in the English vocabulary
    for word in words_in_text:
        if word in english_vocab:  # Check if the word is an English word
            word_count[word] += 1

# Convert the dictionary to a list of tuples (word, count), and sort by count
word_count_list = sorted(word_count.items(), key=lambda x: x[1], reverse=True)

# Select the top n most frequent words
n = 40  
top_n_words = word_count_list[:n]

print(top_n_words)
# Assume X_reduced is the (n_words, 2) array after PCA reduction to 2D
plt.figure(figsize=(10, 10))

# Extract word indexes from vocabulary
words = [word for word, count in top_n_words]
indices = [vocabulary[word] for word in words]  # Get the indexes of top_n_words from the vocabulary

plt.scatter(X_reduced[indices, 0], X_reduced[indices, 1], s=10, cmap='viridis')

# Annotate the points with words from top_n_words
for word, idx in zip(words, indices):
    plt.annotate(word, (X_reduced[idx, 0], X_reduced[idx, 1]))

# Add labels and title
plt.title('2D PCA Word Embedding Visualization for Top n Words')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')

# Save and show the plot
plt.savefig('word_embeddings_pca.png', dpi=300)
plt.show()

plot the clusters in the dimensionality reduction

Heare we chek varins, for chosing the best number of components
Heare we use elbow metod for disede with number of clusters we will chose

In [ ]:
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
from sklearn.preprocessing import StandardScaler

# Extract embeddings from the trained model and standardize them
embedding_layer = skipgram_model.get_layer('embedding')
X = embeddings = embedding_layer.get_weights()[0]  # Extract embeddings
words = vocabulary  # Dictionary with word indexes

# Standardize the data
X_scaled = StandardScaler().fit_transform(X)

# Dimensionality reduction using PCA
n_components = 200  # Number of components to retain 90-95% of the variance
pca = PCA(n_components=n_components)
X_reduced = pca.fit_transform(X_scaled)

# Check explained variance
explained_variance = pca.explained_variance_ratio_.sum()
print(f'Explained variance with {n_components} components: {explained_variance:.2f}')

# Apply KMeans for clustering
n_clusters = 2  # Number of clusters 
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
cluster_labels = kmeans.fit_predict(X_reduced)

# Evaluate clustering quality using metrics
sil_score = silhouette_score(X_reduced, cluster_labels)
ch_score = calinski_harabasz_score(X_reduced, cluster_labels)
db_score = davies_bouldin_score(X_reduced, cluster_labels)

print(f'Silhouette Score: {sil_score}')
print(f'Calinski-Harabasz Index: {ch_score}')
print(f'Davies-Bouldin Index: {db_score}')

# Visualize clusters in 2D
plt.figure(figsize=(10, 10))
scatter = plt.scatter(X_reduced[:, 0], X_reduced[:, 1], c=cluster_labels, cmap='viridis', s=20)
plt.colorbar(scatter, label='Cluster Labels')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.title('Cluster Visualization')
plt.grid(True)

# Annotate some words on the plot
words_to_annotate = ['hotel', 'property', 'good', 'room']  # Words for annotation
for word in words_to_annotate:
    if word in vocabulary:  # Check if the word is in the vocabulary
        idx = vocabulary[word]
        plt.annotate(word, (X_reduced[idx, 0], X_reduced[idx, 1]), fontsize=12)

plt.show()

# Elbow method to determine the optimal number of clusters
distortions = []
for k in range(2, 10):
    kmeans = KMeans(n_clusters=k, random_state=42).fit(X_reduced)
    distortions.append(kmeans.inertia_)

plt.figure(figsize=(8, 6))
plt.plot(range(2, 10), distortions, marker='o')
plt.xlabel('Number of Clusters')
plt.ylabel('Inertia')
plt.title('Elbow Method for Optimal k')
plt.grid(True)
plt.show()